# Metric (Lasot and vot)

## 1.1 Success and Precsion

In [11]:
import math
import numpy as np
from typing import List, Iterable, Tuple, Any
import itertools
import sys
sys.path.append('/home/lz/anaconda3/envs/nbconda/lib/python3.9/site-packages/vot/')
from attributee import Float, Integer, Boolean, Include
import os
from vot.tracker import Tracker
from vot.dataset import Sequence
from vot.region import Region, RegionType, calculate_overlaps
from vot.experiment import Experiment
from vot.experiment.multirun import UnsupervisedExperiment
from vot.analysis import SequenceAggregator, Analysis, SeparableAnalysis, \
    MissingResultsException, Measure, Sorting, Curve, Plot, SequenceAggregator, \
    Axes, analysis_registry
from vot.utilities.data import Grid
import numpy as np
from shapely.geometry import Polygon


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count




def estimateIOU(box_a, box_b, dim=2):
    areaA = box_a[2] * box_a[3]
    areaB = box_b[2] * box_b[3]

    area_sum = areaA + areaB
    max_a_x = box_a[0] + box_a[2]
    max_a_y = box_a[1] + box_a[3]

    max_b_x = box_b[0] + box_b[2]
    max_b_y = box_b[1] + box_b[3]

    inter_x_max = min(max_a_x, max_b_x)
    inter_y_max = min(max_a_y, max_b_y)
    inter_x_min = max(box_a[0], box_b[0])
    inter_y_min = max(box_a[1], box_b[1])
    inter_w = inter_x_max - inter_x_min
    inter_h = inter_y_max - inter_y_min
    if inter_w<=0 or inter_h<=0:
        inter_area = 0 
    else:
        inter_area = inter_w * inter_h

    overlap = inter_area  / (area_sum - inter_area)

    return overlap




class Success(object):
    """Computes and stores the Success"""

    def __init__(self, n=21, max_overlap=1):
        self.max_overlap = max_overlap
        self.Xaxis = np.linspace(0, self.max_overlap, n)
        self.reset()

    def reset(self):
        self.overlaps = []

    def add_overlap(self, val):
        self.overlaps.append(val)

    @property
    def count(self):
        return len(self.overlaps)

    @property
    def value(self):
        # succ = [
        #     np.sum(i >= thres
        #            for i in self.overlaps).astype(float) / self.count
        #     for thres in self.Xaxis
        # ]
        succ = [
            np.sum(np.fromiter((i >= thres
                   for i in self.overlaps), dtype=float)) / self.count
            for thres in self.Xaxis
        ]
        
    
        return np.array(succ)

    @property
    def average(self):
        if len(self.overlaps) == 0:
            return 0
        return np.trapz(self.value, x=self.Xaxis) * 100 / self.max_overlap


class Sequence_t(object):

    def __init__(self, name: str, dataset='/data1/yjy/rgbd_benchamrk/alldata/'):
        self._name = name
        self._dataset = dataset
        self._path = self._dataset + self._name


    @property
    def name(self) -> str:
        return self._name

    @property
    def identifier(self) -> str:
        return self._name

    @property
    def dataset(self):
        return self._dataset

    @property
    def gt(self):
        gtfile = os.path.join(self._path, 'groundtruth.txt')
        with open(gtfile, 'r') as f:
            value = np.loadtxt(f, delimiter=',')
        return value 
    @property
    def invisible(self):
        full_occlusion = os.path.join(self._path, 'full-occlusion.tag')
        with open(full_occlusion, 'r') as f:
            value = np.loadtxt(f)
        return value

class tracking(object):
    def __init__(self, name: str, path='/data1/yjy/rgbd_benchmark/all_benchmark/results'):
        self._name = name
        self._path = os.path.join(path, self._name) 
    
    @property
    def name(self) -> str:
        return self._name
    
    
    def prebox_conf(self, sequence):
        boxtxt = os.path.join(self._path, '{}_001.txt'.format(sequence))
        with open(boxtxt, 'r') as f:
            pre_value = np.loadtxt(f, delimiter=',')
        
        conftxt = os.path.join(self._path, '{}_001_confidence.value'.format(sequence))
        if not os.path.exists(conftxt):
            conftxt = os.path.join(self._path, '{}_001_confidence.txt'.format(sequence))
        if not os.path.exists(conftxt):
            confidence = [1 for i in range(len(pre_value))]
        return pre_value, confidence
    









## 1.2 precision and recall

In [ ]:
class Precision(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
class Recall(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def compute_tpr_curves(trajectory: List, confidence: List[float], sequence: Sequence, thresholds: List[float],
    ignore_unknown: bool = True, bounded: bool = True):

    #overlaps = np.array(calculate_overlaps(trajectory, sequence.groundtruth(), (sequence.size) if bounded else None))
    overlaps = np.array([estimateIOU(trajectory[i], sequence[i] ) for i in range(len(trajectory))])

    confidence = np.array(confidence)

    n_visible = len([region for region in sequence.groundtruth() if region.type is not RegionType.SPECIAL])



    precision = len(thresholds) * [float(0)]
    recall = len(thresholds) * [float(0)]

    for i, threshold in enumerate(thresholds):

        subset = confidence >= threshold

        if np.sum(subset) == 0:
            precision[i] = 1
            recall[i] = 0
        else:
            precision[i] = np.mean(overlaps[subset])
            recall[i] = np.sum(overlaps[subset]) / n_visible

    return precision, recall

### 1 read full-occlusion tag 

#### 2 read confidence score and tracking result (s)

## calculate auc of iou 

In [30]:
import os
# tracker_name = ['DDiMP', 'ATCAIS', 'DRefine', 'SLMD', 'Siam_LTD', 'TSDM', 'iiau_rgbd']
tracker_name = ['ATCAIS19']
gt_dir = '/data1/yjy/rgbd_benchmark/stc_benchmark/RGBDdataset/'
su = Success()
for tracker in tracker_name:
    su.reset()
    tracking_result_dir = '/home/yangjinyu/rgbd_tracker/benchmark_workspace/stc_workspace/results/{}/rgbd-unsupervised'.format(tracker)
    seqlist = os.listdir(tracking_result_dir)
    for seq in seqlist:
        gt_seq = os.path.join(gt_dir, seq, 'groundtruth.txt' )
        pre_seq = os.path.join(tracking_result_dir, '{}/{}_001.txt'.format(seq, seq))
        with open(gt_seq, 'r') as f:
            gt_value = np.loadtxt(f, delimiter=',', skiprows=1)
        with open(pre_seq, 'r') as f:
            pre_value = np.loadtxt(f, delimiter=',', skiprows=1)
        if not len(gt_value) == len(pre_value):
            raise TypeError
        for i in range(len(gt_value)):
            iou = estimateIOU(gt_value[i], pre_value[i])
            su.add_overlap(iou)
    print('{} iou auc: {}'.format(tracker, su.average))
    


/tmp/ipykernel_35807/2209188759.py:76: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  np.sum(i >= thres


ATCAIS19 iou auc: 66.43529551954242


### stc tracker

In [32]:
import os
# tracker_name = ['DDiMP', 'ATCAIS', 'DRefine', 'SLMD', 'Siam_LTD', 'TSDM', 'iiau_rgbd']
tracker_name = ['stc']
gt_dir = '/data1/yjy/rgbd_benchmark/stc_benchmark/RGBDdataset/'
su = Success()
for tracker in tracker_name:
    su.reset()
    tracking_result_dir = '/home/yangjinyu/rgbd_tracker/benchmark_workspace/stc_workspace/results/{}/rgbd-unsupervised'.format(tracker)
    seqlist = os.listdir(tracking_result_dir)
    for seqtxt in seqlist:
        gt_seq = os.path.join(gt_dir, seqtxt.split('.')[0], 'groundtruth.txt' )
        pre_seq = os.path.join(tracking_result_dir, seqtxt)
        with open(gt_seq, 'r') as f:
            gt_value = np.loadtxt(f, delimiter=',', skiprows=1)
        with open(pre_seq, 'r') as f:
            pre_value = np.loadtxt(f, delimiter=',', skiprows=1)
        if not len(gt_value) == len(pre_value):
            raise TypeError
        for i in range(len(gt_value)):
            iou = estimateIOU(gt_value[i], pre_value[i])
            su.add_overlap(iou)
    print('{} iou auc: {}'.format(tracker, su.average))

stc iou auc: 39.79200786112021


### copy result to all_benchmark

In [9]:
import os
import shutil
import distutils.dir_util
all_benchmark = '/data1/lz/results/'
depthtrack_benchmark = '/home/yangjinyu/rgbd_tracker/benchmark_workspace/depthtrack_workspace/results/'
stc_benchmark = '/home/yangjinyu/rgbd_tracker/benchmark_workspace/stc_workspace/results/'
ptb_benchmark = '/home/yangjinyu/rgbd_tracker/benchmark_workspace/ptb_workspace/results/'
cdtb_benchmark = '/data1/yjy/workspace_votRGBD2019/workspace/results/'

# result = os.listdir(depthtrack_benchmark)
distutils.dir_util.copy_tree(cdtb_benchmark, all_benchmark)

DistutilsFileError: could not open '/data1/yjy/workspace_votRGBD2019/workspace/results/iiau_rgbd/rgbd-unsupervised/box_darkroom_noocc_6/box_darkroom_noocc_6_001_time.value': Permission denied